## 主題：以CNN做表情辨認

### 組員：張毓倩、江育誠、黃涱煜

### 介紹：
   
    本資料來自於 kaggle 網站(來源在文末連結)，共有兩萬八千張左右48x48 pixel的圖片，每一張圖片皆有表情label，而每張圖片都只會有一種表情。總共有七種可能的表情（0：生氣, 1：厭惡, 2：恐懼, 3：高興, 4：難過, 5：驚訝, 6：中立(難以區分為前六種的表情))。此外，還有七千筆 testing data 是沒有 label 的，因此我們會將兩萬八千張有 label 的資料以 CNN 建立出預測模型後，再預測出 testing data 的 label。最後上傳至 kaggle 查看預測準確率。
    (https://www.kaggle.com/c/ml2019spring-hw3/kernels)
  

### 實作方法：

    1. Training and validation
    
        將兩萬八千筆有 label 的資料切割成 training set 與 validation set，以 CNN 的方式建模並以 validation set 的 accuracy 當作預測指標。當 accuracy 符合預期後，以同樣的 model 參數再用全部兩萬八千筆資料來 fit，得到最終的 model。
    
    2. Testing
    
        將最終的 model 預測那七千筆 testing data 的 label，上傳至 kaggle 查看預測準確率。

### 專案排程

    5/26設計CNN模型
    
    5/28使用較少量資料跑模型
    
    6/2比較各模型結果並選定模型
    
    6/3~6/6彙整並撰寫期末專題報告

### 目前進度：以部分資料建立模型並評估

    從兩萬八千筆有 label 的資料中，挑選前 5000 筆當作 training set，最後 1500 筆做 testing set，以少量資料初步評估模型的預測率。

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, \
      Activation, BatchNormalization
from keras.utils import np_utils
from keras.optimizers import adam

Using TensorFlow backend.


In [3]:
def load_data():
   all_features = []
   all_labels = []
   count = 0
   test_num = 5000
   with open("train.csv", 'r', encoding='big5') as file:
      file.readline()
      all_rows = csv.reader(file, delimiter=',')
      for row in all_rows:
         if count == test_num:
            break
         all_labels.append(row[0])
         all_features.append(row[1].split(' '))
         count += 1

   all_features = np.array(all_features).astype('float')
   all_features = all_features.reshape(test_num, 48, 48, 1)

   all_labels = np.array(all_labels).astype('float')
   all_labels = np_utils.to_categorical(all_labels, 7)

   return all_features, all_labels

In [2]:
def load_data2():
   all_features = []
   all_labels = []
   count = 0
   test_num = 1500
   with open("validation.csv", 'r', encoding='big5') as file:
      file.readline()
      all_rows = csv.reader(file, delimiter=',')
      for row in all_rows:
         if count == test_num:
            break
         all_labels.append(row[0])
         all_features.append(row[1].split(' '))
         count += 1

   all_features = np.array(all_features).astype('float')
   all_features = all_features.reshape(test_num, 48, 48, 1)

   all_labels = np.array(all_labels).astype('float')
   all_labels = np_utils.to_categorical(all_labels, 7)

   return all_features, all_labels

In [4]:
def build_model(x_train):
   model = Sequential()
   num_classes = 7

   model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(Conv2D(64, (3, 3)))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(MaxPool2D(pool_size=(2, 2)))

   model.add(Conv2D(32, (3, 3), padding='same'))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(Conv2D(32, (3, 3)))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(MaxPool2D(pool_size=(2, 2)))

   model.add(Flatten())

   model.add(Dense(16))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(Dense(64))
   model.add(BatchNormalization())
   model.add(Activation('relu'))

   model.add(Dense(num_classes))
   model.add(Activation('softmax'))

   return model

In [5]:
## load data ##
X_train, Y_train = load_data()

## normalize ##
X_train = X_train / 255

In [6]:
## load data ##
X_test, Y_test = load_data()

## normalize ##
X_test = X_test / 255

In [7]:
## main ##
model = build_model(X_train)
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=0.001),
               metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 46, 46, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
__________

In [9]:
model.fit(X_train, Y_train, batch_size=256, epochs=20,validation_data = (X_test, Y_test))

Train on 5000 samples, validate on 5000 samples
Epoch 1/20
5000/5000 [==============================] - 176s 35ms/step - loss: 1.6780 - acc: 0.3644 - val_loss: 1.8721 - val_acc: 0.2564
Epoch 2/20
5000/5000 [==============================] - 177s 35ms/step - loss: 1.4875 - acc: 0.4534 - val_loss: 1.8541 - val_acc: 0.2620
Epoch 3/20
5000/5000 [==============================] - 176s 35ms/step - loss: 1.3312 - acc: 0.5140 - val_loss: 1.8415 - val_acc: 0.2598
Epoch 4/20
5000/5000 [==============================] - 177s 35ms/step - loss: 1.1977 - acc: 0.5752 - val_loss: 1.8515 - val_acc: 0.2298
Epoch 5/20
5000/5000 [==============================] - 176s 35ms/step - loss: 1.0664 - acc: 0.6254 - val_loss: 1.8690 - val_acc: 0.2268
Epoch 6/20
5000/5000 [==============================] - 177s 35ms/step - loss: 0.9500 - acc: 0.6758 - val_loss: 1.8813 - val_acc: 0.1920
Epoch 7/20
5000/5000 [==============================] - 176s 35ms/step - loss: 0.8144 - acc: 0.7338 - val_loss: 1.9799 - val_acc: 

In [10]:
model.save('model-5000.h5')

### 目前進度結論：

    在模型參數高達10萬多個下，Training set accuracy = 99.66%，但 testing set accuracy = 34.22%，因此以目前看來該模型的設計可能有 overfitting 的現象，之後設計模型的方向會以避免過度擬合的角度去發展。